In [1]:
import numpy as np
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, LSTM, Input, concatenate
from keras.optimizers import Adadelta
from keras.callbacks import ModelCheckpoint
import pandas as pd
import csv
from sklearn.metrics import mean_squared_error
import ast
import operator
import datetime
from sklearn.preprocessing import MinMaxScaler
import random
from sklearn.metrics import mean_squared_error

/Users/huymac/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
zip_dictionary = {'San Francisco': 94107,
                 'Redwood City': 94063,
                 'Palo Alto': 94301,
                 'Mountain View': 94041,
                 'San Jose': 95113}

In [3]:
"""read station_data.csv file

returns
    station_dic {id: [lat, long, dock count,zip]}
    
"""
station_dic = {}
with open("data/station_data.csv","r") as station_file:
    station_file.readline() #skip the first line
    for line in station_file:
        line = line.replace("\r\n","")
        fields = line.split(",")
        station_dic[int(fields[0])] = []
        for i in range(2,5):
            station_dic[int(fields[0])].append(ast.literal_eval(fields[i]))
        
        station_dic[int(fields[0])].append(zip_dictionary[fields[5]])
print ("Example of station id 5:", station_dic[5])
        


('Example of station id 5:', [37.331415, -121.8932, 19, 95113])


In [4]:
"""read weather_data.csv file

returns
    weather_dic {(date, zip): [Max Temperature, Mean Temperature, ...]}
"""

weather_dic = {}
events_dic = {'': [1,0,0,0,0], 'Rain': [0,1,0,0,0], 'Fog-Rain': [0,0,1,0,0], 'Rain-Thunderstorm': [0,0,0,1,0], 'Fog': [0,0,0,0,1]}
with open("data/weather_data.csv","r") as weather_file:
    weather_file.readline() #skip the first line
    for line in weather_file:
        line = line.replace("\r\n","")
        fields = line.split(",")
        weather_dic[(fields[0],int(fields[-1]))]= []
        for i in range(1,len(fields)-1):
            if i == (len(fields)-3):
                weather_dic[(fields[0],int(fields[-1]))] += events_dic[fields[i]]
            elif fields[i]=="":
                weather_dic[(fields[0],int(fields[-1]))].append(0)
            else:
                weather_dic[(fields[0],int(fields[-1]))].append(ast.literal_eval(fields[i]))
print ("Example of weather at (17/09/2014,94107):",weather_dic[("17/09/2014",94107)])
        

('Example of weather at (17/09/2014,94107):', [77, 69, 61, 64, 58, 55, 84, 67, 50, 29.83, 29.8, 29.75, 10, 10, 9, 21, 9, 29, 0.001, 5, 0, 1, 0, 0, 0, 267])


In [5]:
"""read trip_data.csv file

returns
    trip_started_dictionary {(date, station id, hour): count}
    trip_ended_dictionary {(date, station id, hour): count}
    net_rate_dictionary {(date, station id, hour): net rate}
    top_route_dictionary {station id: [station id1, station id2,...]}
    route_dictionary {(station1,station2):count}
"""
trip_started_dictionary = {}
trip_ended_dictionary = {}
net_rate_dictionary = {}
top_route_dictionary = {}
route_dictionary = {}
date_list = []
type_dic = {"Subscriber":0, "Customer":1}
count = 0
with open("data/trip_data.csv","r") as trip_file:
    trip_file.readline() #skip the first line
    for line in trip_file:
        line = line.replace("\n","").replace("\r","")
        fields = line.split(",")
        start_date = fields[1].split(" ")[0]
        start_station = int(fields[2])
        start_hour = int(fields[1].split(" ")[1].split(":")[0])
        end_date = fields[3].split(" ")[0]
        end_station =  int(fields[4])
        end_hour = int(fields[3].split(" ")[1].split(":")[0])
        sub_type = type_dic[fields[-1]]

        #print (date,start_station,start_hour,end_station,end_hour,sub_type)
        if (start_date,start_station,start_hour) in trip_started_dictionary:
            trip_started_dictionary[(start_date,start_station,start_hour)] +=1
        else:
            trip_started_dictionary[(start_date,start_station,start_hour)] = 1
        if (start_date,start_station,start_hour) not in trip_ended_dictionary:
            trip_ended_dictionary[(start_date,start_station,start_hour)] = 0
        if (end_date,end_station,end_hour) in trip_ended_dictionary:
            trip_ended_dictionary[(end_date,end_station,end_hour)] +=1
        else:
            trip_ended_dictionary[(end_date,end_station,end_hour)] =1
            
        if (end_date,end_station,end_hour) not in trip_started_dictionary:
            trip_started_dictionary[(end_date,end_station,end_hour)] = 0
        if (start_station,end_station) not in route_dictionary:
            route_dictionary[(start_station,end_station)] = 1
            route_dictionary[(end_station,start_station)] = 1
        else:
            route_dictionary[(start_station,end_station)] += 1
            route_dictionary[(end_station,start_station)] += 1
        
        
for station1 in station_dic:
    station1_route = {}
    for station2 in station_dic:
        if (station1,station2) in route_dictionary:
            station1_route[station2] = route_dictionary[(station1,station2)]
    station1_route = sorted(station1_route.items(),key=operator.itemgetter(1),reverse=True)
    top_route_dictionary[station1] = station1_route 

     

for key in trip_ended_dictionary:
    net_rate_dictionary[key] = trip_ended_dictionary[key] - trip_started_dictionary[key]
print("Example of net rate dictionary:", dict(list(net_rate_dictionary.items())[0:5]))
print("Example of top route dictionary:", dict(list(top_route_dictionary.items())[0:5]))
        

('Example of net rate dictionary:', {('31/03/2015', 9, 11): 1, ('20/07/2015', 32, 6): -1, ('27/04/2015', 4, 13): 1, ('02/07/2015', 74, 14): 0, ('15/07/2015', 90, 7): -1})
('Example of top route dictionary:', {2: [(4, 2446), (6, 1301), (11, 1129), (7, 790), (13, 653), (84, 645), (9, 611), (5, 575), (10, 518), (3, 491), (16, 229), (8, 215), (12, 206), (2, 153), (80, 25), (14, 23), (28, 2)], 3: [(2, 491), (3, 377), (10, 133), (84, 122), (6, 100), (11, 69), (12, 67), (4, 63), (7, 59), (9, 43), (5, 39), (14, 39), (16, 35), (8, 19), (80, 14), (13, 9), (28, 2), (29, 1)], 4: [(2, 2446), (14, 510), (12, 305), (4, 209), (7, 65), (3, 63), (8, 37), (11, 30), (5, 29), (10, 29), (6, 27), (16, 26), (9, 21), (84, 17), (13, 13), (80, 8)], 5: [(2, 575), (12, 60), (5, 57), (7, 56), (6, 42), (3, 39), (10, 32), (4, 29), (11, 29), (13, 25), (9, 24), (8, 23), (16, 17), (14, 15), (84, 15), (80, 13)], 6: [(2, 1301), (9, 247), (84, 218), (80, 202), (13, 176), (6, 171), (14, 137), (3, 100), (8, 83), (10, 80), (7

In [6]:
""" Input: station_data_array, conntectedStation_data_array and Output y_data preprocessing"""

weekday_dic = {"Monday":[1,0,0,0,0,0,0], "Tuesday":[0,1,0,0,0,0,0], "Wednesday":[0,0,1,0,0,0,0],
              "Thursday":[0,0,0,1,0,0,0], "Friday":[0,0,0,0,1,0,0], "Saturday":[0,0,0,0,0,1,0], "Sunday":[0,0,0,0,0,0,1]}
season_dic = {"01":[1,0,0,0],"02":[1,0,0,0],"03":[1,0,0,0],"04":[0,1,0,0],"05":[0,1,0,0],"06":[0,1,0,0],"07":[0,0,1,0],
             "08":[0,0,1,0],"09":[0,0,1,0],"10":[0,0,0,1],"11":[0,0,0,1],"12":[0,0,0,1]}
station_data_array = []
conntectedStation_data_array = []
y_data = []
full_data = []
for item in net_rate_dictionary:
    date, station_id, hour = item
    label = net_rate_dictionary[item]
    lat, longtitue, dockcount,zip_code  = station_dic[station_id]
    weather = weather_dic[(date,zip_code)]
    dd,mm,yy = date.split("/")
    weekday = weekday_dic[datetime.date(int(yy),int(mm),int(dd)).strftime("%A")]
    season = season_dic[mm]
    hour_category = [0]*24
    hour_category[hour] = 1
    station_data = [lat,longtitue,dockcount]+weekday+season+weather+hour_category
    #Station infor
    station_data_array.append(station_data)
    #Top 5 connected station in previous three hours
    connected_stations = top_route_dictionary[station_id][0:5]
    connected_station_data = []
    for pre_hour in range(hour-3,hour):
        connected_station_data_perhour = []
        if pre_hour < 0:
            pre_hour += 24
            current_date = (datetime.date(int(yy),int(mm),int(dd)) - datetime.timedelta(1)).strftime("%d/%m/%Y")
        else:
            current_date = date
        for connected_station_id,value in connected_stations:
            if (current_date,connected_station_id,pre_hour) not in net_rate_dictionary:
                start = 0
                end = 0
                net = 0
            else:
                start = trip_started_dictionary[(current_date,connected_station_id,pre_hour)]
                end = trip_ended_dictionary[(current_date,connected_station_id,pre_hour)]
                net = net_rate_dictionary[(current_date,connected_station_id,pre_hour)]
            connected_station_data_perhour.append(start)
            connected_station_data_perhour.append(end)
            connected_station_data_perhour.append(net)
        #append infor of the current station
        if (current_date,station_id,pre_hour) not in net_rate_dictionary:
            start = 0
            end = 0
            net = 0
        else:
            start = trip_started_dictionary[(current_date,station_id,pre_hour)]
            end = trip_ended_dictionary[(current_date,station_id,pre_hour)]
            net = net_rate_dictionary[(current_date,station_id,pre_hour)]
        connected_station_data_perhour.append(start)
        connected_station_data_perhour.append(end)
        connected_station_data_perhour.append(net)
        
        connected_station_data.append(connected_station_data_perhour)
    conntectedStation_data_array.append(connected_station_data)
    y_data.append([label])
    full_data.append([date,station_id,hour,weather])

    

In [7]:
"""Normalize data"""
station_data_scaler = MinMaxScaler()
station_data_scaler.fit(station_data_array)
station_data_norm_array = station_data_scaler.transform(station_data_array)


y_data_scaler = MinMaxScaler(feature_range=(-20,20))
y_data_scaler.fit(y_data)
y_data_norm_array = y_data_scaler.transform(y_data)
conntectedStation_data_array = np.array(conntectedStation_data_array)
conntectedStation_data_array = conntectedStation_data_array.reshape(-1,54)

conntectedStation_data_scaler = MinMaxScaler()
conntectedStation_data_scaler.fit(conntectedStation_data_array)
conntectedStation_data_array = conntectedStation_data_scaler.transform(conntectedStation_data_array)

conntectedStation_data_array = conntectedStation_data_array.reshape(-1,3,18)


/Users/huymac/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [8]:
"""Split data into train/dev/test"""
station_data_norm_array = np.array(station_data_norm_array)
y_data_norm_array = np.array(y_data_norm_array)


station_data_norm_array_train = []
station_data_norm_array_dev = []
station_data_norm_array_test = []

conntectedStation_data_array_train = []
conntectedStation_data_array_dev = []
conntectedStation_data_array_test = []

y_data_norm_array_train = []
y_data_norm_array_dev = []
y_data_norm_array_test = []

full_data_test = []

#Split data for train/dev/test 60/20/20
for i in range(0,len(y_data_norm_array)):
    randnum = random.randint(0,10)
    if randnum <= 6:
        station_data_norm_array_train.append(station_data_norm_array[i])
        conntectedStation_data_array_train.append(conntectedStation_data_array[i])
        y_data_norm_array_train.append(y_data_norm_array[i])
    elif randnum >6 and randnum <=8:
        station_data_norm_array_dev.append(station_data_norm_array[i])
        conntectedStation_data_array_dev.append(conntectedStation_data_array[i])
        y_data_norm_array_dev.append(y_data_norm_array[i])
    else:
        station_data_norm_array_test.append(station_data_norm_array[i])
        conntectedStation_data_array_test.append(conntectedStation_data_array[i])
        y_data_norm_array_test.append(y_data_norm_array[i])
        full_data_test.append(full_data[i])
        
station_data_norm_array_train = np.array(station_data_norm_array_train)
station_data_norm_array_dev = np.array(station_data_norm_array_dev)
station_data_norm_array_test = np.array(station_data_norm_array_test)

conntectedStation_data_array_train = np.array(conntectedStation_data_array_train)
conntectedStation_data_array_dev = np.array(conntectedStation_data_array_dev)
conntectedStation_data_array_test = np.array(conntectedStation_data_array_test)

y_data_norm_array_train = np.array(y_data_norm_array_train)
y_data_norm_array_dev = np.array(y_data_norm_array_dev)
y_data_norm_array_test = np.array(y_data_norm_array_test)

print ("Shape of station data train:",station_data_norm_array_train.shape)
print ("Shape of Connected stations data train:",conntectedStation_data_array_train.shape)
print ("Shape of output train:",y_data_norm_array_train.shape)



print ("Size of train:",len(y_data_norm_array_train))
print ("Size of test:",len(y_data_norm_array_test))
print ("Size of dev:",len(y_data_norm_array_dev))

('Shape of station data train:', (126184, 64))
('Shape of Connected stations data train:', (126184, 3, 18))
('Shape of output train:', (126184, 1))
('Size of train:', 126184)
('Size of test:', 36107)
('Size of dev:', 35865)


In [9]:
"""Build model"""
model = Sequential()

input_station_info_layer = Input(shape=(64,),dtype='float32',name="station_info_input")
input_connected_station_info_layer = Input(shape=(3,18),dtype='float32',name="connected_station_info_input")

hidden_lstm = LSTM(64)(input_connected_station_info_layer)
hidden_dense = Dense(32,activation="sigmoid")(input_station_info_layer)

merge_layer = concatenate([hidden_lstm,hidden_dense],axis=1)
out_layer = Dense(1,activation="linear")(merge_layer)

model = Model(input=[input_station_info_layer,input_connected_station_info_layer], output=[out_layer])
adadelta = Adadelta(lr=1.0, rho=0.95, epsilon=1e-06)
checkpoint = ModelCheckpoint("best.hdf5",monitor="val_loss",mode="min", save_best_only=True)
model.compile(loss="mean_squared_error", optimizer=adadelta)

model.fit([station_data_norm_array_train,conntectedStation_data_array_train],y_data_norm_array_train,
         batch_size=100,nb_epoch=100, callbacks=[checkpoint],
         validation_data=([station_data_norm_array_dev,conntectedStation_data_array_dev],y_data_norm_array_dev))



W0815 07:19:04.358596 4698949056 deprecation_wrapper.py:119] From /Users/huymac/anaconda2/lib/python2.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0815 07:19:04.385994 4698949056 deprecation_wrapper.py:119] From /Users/huymac/anaconda2/lib/python2.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0815 07:19:04.401127 4698949056 deprecation_wrapper.py:119] From /Users/huymac/anaconda2/lib/python2.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

/Users/huymac/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=[<tf.Tenso..., inputs=[<tf.Tenso...)`
  del sys.path[0]
W0815 07:19:04.691793 4698949056 deprecation_wrapper.

Train on 126184 samples, validate on 35865 samples
Epoch 1/100
126184/126184 [==============================] - 9s 73us/step - loss: 1.6365 - val_loss: 1.5677
Epoch 2/100
126184/126184 [==============================] - 10s 78us/step - loss: 1.4768 - val_loss: 1.5037
Epoch 3/100
126184/126184 [==============================] - 9s 69us/step - loss: 1.4138 - val_loss: 1.4276
Epoch 4/100
126184/126184 [==============================] - 8s 60us/step - loss: 1.3912 - val_loss: 1.4296
Epoch 5/100
126184/126184 [==============================] - 8s 62us/step - loss: 1.3704 - val_loss: 1.3960
Epoch 6/100
126184/126184 [==============================] - 8s 64us/step - loss: 1.3516 - val_loss: 1.4519
Epoch 7/100
126184/126184 [==============================] - 8s 61us/step - loss: 1.3295 - val_loss: 1.3400
Epoch 8/100
126184/126184 [==============================] - 8s 63us/step - loss: 1.3124 - val_loss: 1.3918
Epoch 9/100
126184/126184 [==============================] - 8s 60us/step - loss: 1.

126184/126184 [==============================] - 8s 64us/step - loss: 0.9822 - val_loss: 1.0792
Epoch 76/100
126184/126184 [==============================] - 8s 64us/step - loss: 0.9802 - val_loss: 1.0716
Epoch 77/100
126184/126184 [==============================] - 8s 65us/step - loss: 0.9782 - val_loss: 1.0583
Epoch 78/100
126184/126184 [==============================] - 8s 64us/step - loss: 0.9770 - val_loss: 1.0698
Epoch 79/100
126184/126184 [==============================] - 8s 63us/step - loss: 0.9755 - val_loss: 1.0613
Epoch 80/100
126184/126184 [==============================] - 9s 70us/step - loss: 0.9733 - val_loss: 1.0667
Epoch 81/100
126184/126184 [==============================] - 9s 71us/step - loss: 0.9703 - val_loss: 1.0762
Epoch 82/100
126184/126184 [==============================] - 8s 64us/step - loss: 0.9685 - val_loss: 1.0649
Epoch 83/100
126184/126184 [==============================] - 8s 60us/step - loss: 0.9659 - val_loss: 1.0669
Epoch 84/100
126184/126184 [====

In [10]:
"""Performance analysis"""
model.load_weights("best.hdf5")
model.compile(loss="mean_squared_error", optimizer=adadelta)

y_predict_norm = model.predict([station_data_norm_array_test,conntectedStation_data_array_test])
y_predict = y_data_scaler.inverse_transform(y_predict_norm)
y_data_test = y_data_scaler.inverse_transform(y_data_norm_array_test)
for i in range(len(y_predict)):
    y_predict[i][0] = round(y_predict[i][0])
#print y_predict[0:20]
#print y_data_test[0:20]
print ("Mean squared error on test set:",mean_squared_error(y_predict, y_data_test))

('Mean squared error on test set:', 5.195668429944332)


In [11]:
"""Error analysis
Report some cases with high error"""
for i in range(len(y_predict)):
    if abs(y_predict[i][0]-y_data_test[i][0]) > 20:
        print i, y_predict[i][0],y_data_test[i][0]
        print full_data_test[i], station_data_norm_array_test[i]
         

492 5.0 28.0
['13/04/2015', 88, 18, [62, 56, 50, 50, 47, 45, 89, 72, 55, 30.17, 30.09, 30.02, 10, 10, 7, 29, 15, 36, 0, 4, 1, 0, 0, 0, 0, 276]] [0.94069948 0.04336001 0.75       1.         0.         0.
 0.         0.         0.         0.         0.         1.
 0.         0.         0.60784314 0.68292683 0.66666667 0.73529412
 0.734375   0.71428571 0.89       0.75       0.63953488 0.9898294
 0.98947715 0.98880105 0.5        0.5        0.35       0.2265625
 0.65217391 0.58064516 0.         0.5        1.         0.
 0.         0.         0.         0.76666667 0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         1.         0.
 0.         0.         0.         0.        ]
693 1.0 25.0
['19/03/2015', 70, 16, [73, 60, 47, 52, 49, 43, 93, 64, 35, 30.08, 30.05, 30, 10, 10, 10, 18, 6, 21, 0, 2, 1, 0, 0, 0, 0, 294]] [0.94073527 0.04374775 0.5        0.         0.         0